In [1]:
import pandas as pd
import os

import warnings
warnings.filterwarnings("ignore")

In [ ]:
daily : pd.DataFrame = pd.read_excel(r"C:\Users\sevan\OneDrive - De Vinci\A4\Pi2\Data\Bloomberg Data.xlsx", sheet_name="Daily", header=5)

daily.drop([0,1,2], inplace=True)

daily.rename(columns={'Unnamed: 0':"Date"}, inplace=True)

daily["Date"] = pd.to_datetime(daily["Date"])

daily.set_index("Date", inplace=True)

In [7]:
daily.columns = ['SPY', 'NSQ', 'CAC', 'GERM10Y', 'F10Y', 'UST10Y', 'BTC', 'ETH',
 'SOL', 'EURUSD', 'USDJPY', 'GBPEUR', 'GOLD', 'BRENT', 'GAS', 'AAPL',
 'MSFT', 'TSLA', 'GS', 'XOM', 'LVMH', 'TTE', 'BNP']

series = {c : daily[c].dropna() for c in daily.columns}

# series["SPY"].head()

for k in series.keys():
    
    series[k].to_excel(rf"C:\Users\sevan\OneDrive - De Vinci\A4\Pi2\Data\Daily\{k}.xlsx")

In [ ]:
minute : pd.DataFrame = pd.read_excel(r"C:\Users\sevan\OneDrive - De Vinci\A4\Pi2\Data\Bloomberg Data.xlsx", sheet_name="1min")

In [ ]:
# minute.drop(0, inplace = True)
df = minute.copy()

In [40]:
new_columns = []

for i, col in enumerate(df.columns):
    if "BarTp" in col:
        # Nom de la colonne en fonction de la première ligne (valeur de la ligne 0)
        new_columns.append(f"{df.iloc[0][col]}")  # Nom de la colonne, basé sur la première ligne
    else:
        # Ajouter un suffixe (Bid ou Ask) et le nom de l'indice (par exemple, SPX Index)
        prefix = df.iloc[0][df.columns[i - 1]] if "BarTp" in df.columns[i - 1] else None
        if prefix:
            new_columns.append(f"Bid {prefix}" if "Bid" in col else f"Ask {prefix}")
        else:
            new_columns.append(col)  # Si la colonne n'est ni "BarTp", ni "Bid" ni "Ask", conserver le nom original

df.columns = new_columns

df.drop([1,2,3], inplace = True)

df.columns.values[0] = "Date"

df["Date"] = pd.to_datetime(df["Date"])

df.set_index(df["Date"], inplace=True)

df = df.filter(regex='^(Bid|Ask)', axis=1)

In [58]:
sub_dfs = []

# Récupérer les indices des colonnes qui commencent par "Bid"
for i, col in enumerate(df.columns):
    if col.startswith("Bid"):
        # Identifier la colonne "Ask" correspondante
        ask_col = col.replace("Bid", "Ask")
        
        # Si la colonne "Ask" existe, créer le sous-DataFrame
        if ask_col in df.columns:
            # Utiliser iloc pour sélectionner les colonnes par position
            bid_col_index = df.columns.get_loc(col)
            ask_col_index = df.columns.get_loc(ask_col)
            
            # Créer le sous-DataFrame avec "Date", "Bid", et "Ask" en utilisant iloc
            sub_df = df.iloc[:, [bid_col_index, ask_col_index]]
            sub_dfs.append(sub_df)

for e in sub_dfs:
    e["Mean Price"] = e.mean(axis=1)

In [78]:
for i, e in enumerate(sub_dfs):

    name = daily.columns[i]

    e.columns.values[2] = name

    e.dropna(inplace=True)

    e = e[e.iloc[:, 2] != 0]

    e.iloc[:, 2].to_excel(f"1min\{name}.xlsx")